In [ ]:
# Copyright 2025 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - Chat history management

* This colab explains how to deal with chat history being stored in firestore.   
* Firestore reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/

## Init Vertex AI SDK

In [ ]:
%pip install --quiet --user google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
%pip install  --user --quiet google-cloud-firestore
%pip install  --user --quiet langchain-google-firestore
%pip install  --user --quiet langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00


#### Note: <font color="red">Need to restart the kernel after installing</font>

In [ ]:
# Restart runtime.
exit()

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
# MODEL_NAME = "gemini-1.5-flash-002"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

In [ ]:
# @title Firestore configuration
DATABASE="chat-history"
COLLECTION="history"

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# @title Initialize Vertex AI with Staging Bucket.
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Firestore setting for chat history

Before executing the following cell, you have to set up firestore database in GCP Console.
* Create database : https://cloud.google.com/firestore/docs/manage-databases

In [ ]:
#@title Helper function to get chat history from firestore.

# Reference from : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    firestore_client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    chat_history = FirestoreChatMessageHistory(
                      client=firestore_client,
                      session_id=session_id,
                      collection=COLLECTION,
                      encode_message=False,
    )

    return chat_history


In [ ]:
#@title Helper functions to interact with Agent.

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

In [ ]:
#@title Unit test for Firestore.
session_id = "conv-004"

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [ ]:
  #@title Crate a tool for Google search.
  from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

  google_search_tool = Tool(
      google_search = GoogleSearch()
  )

### Agent - Local unit test

In [ ]:
from vertexai import agent_engines

# Agent Define : https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.LangchainAgent#vertexai_preview_reasoning_engines_LangchainAgent

local_agent = agent_engines.LangchainAgent(
    model=MODEL_NAME,
    # tools=[google_search_tool,],  # Need to check why Google search isn't supported. error message:  Unsupported tool call type function_declarations=None retrieval=None google_search=GoogleSearch() google_search_retrieval=None code_execution=None
    chat_history=get_session_history,  # function to get FirestoreChatMessageHistory
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [ ]:
from time import perf_counter

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(local_agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")


사용자: 안녕하세요.


1.469619839999723

AI Agent : 안녕하세요! 무엇을 도와드릴까요? 😊


------------------------------------ 
사용자: 생성형 AI 부정적인 측면 알려주세요.


6.607728229999793

AI Agent : 생성형 AI는 놀라운 가능성을 보여주지만, 동시에 여러 가지 부정적인 측면도 가지고 있습니다. 주요한 부정적 측면들을 자세히 알려드릴게요.

**1. 허위 정보 및 가짜 콘텐츠 생성:**

*   **가짜 뉴스 및 허위 정보 확산:** 생성형 AI는 매우 사실적인 텍스트, 이미지, 오디오, 비디오를 만들 수 있기 때문에 가짜 뉴스를 쉽게 생성하고 퍼뜨릴 수 있습니다. 이는 사회적 혼란과 잘못된 정보에 기반한 의사 결정을 야기할 수 있습니다.
*   **딥페이크:** 특정 인물의 얼굴이나 목소리를 합성하여 조작된 콘텐츠를 만드는 딥페이크는 명예훼손, 사기, 정치적 선전 등 다양한 악용 사례를 만들 수 있습니다.

**2. 일자리 감소 및 경제적 불평등 심화:**

*   **자동화로 인한 실업:** 생성형 AI는 콘텐츠 제작, 데이터 분석, 고객 서비스 등 다양한 분야에서 인간의 역할을 대체할 수 있습니다. 이는 특정 직업군의 실업률 증가로 이어질 수 있습니다.
*   **숙련 노동자와 비숙련 노동자 간 격차 심화:** AI 기술을 활용할 수 있는 숙련 노동자와 그렇지 못한 비숙련 노동자 간의 임금 격차가 커질 수 있습니다.

**3. 저작권 및 지적 재산권 침해:**

*   **저작권 침해 콘텐츠 생성:** 생성형 AI는 학습 데이터에 포함된 저작물을 무단으로 사용하여 새로운 콘텐츠를 생성할 수 있습니다. 이는 저작권자의 권리를 침해하고 창작 생태계를 위협할 수 있습니다.
*   **표절 문제:** 생성형 AI가 생성한 콘텐츠가 기존 작품과 유사하거나 표절 시비에 휘말릴 가능성이 있습니다.

**4. 편향성 및 차별 심화:**

*   **학습 데이터 편향으로 인한 차별적 결과:** 생성형 AI는 학습 데이터에 존재하는 편향을 그대로 반영하여 성별, 인종, 종교 등에 대한 차별적인 결과를 낼 수 있습니다.
*   **사회적 불평등 심화:** AI 시스템이 특정 집단에 불리한 결정을 내리거나 자원을 불공정하게 분배함으로써 사회적 불평등을 심화시킬 수 있습니다.

**5. 개인 정보 침해 및 감시 강화:**

*   **개인 정보 수집 및 악용:** 생성형 AI는 사용자의 데이터를 수집하고 분석하여 개인의 성향, 취향, 행동 패턴 등을 파악할 수 있습니다. 이러한 정보는 광고, 마케팅, 정치적 선전 등 다양한 목적으로 악용될 수 있습니다.
*   **감시 기술 악용:** 생성형 AI는 얼굴 인식, 음성 인식 등 감시 기술과 결합되어 개인의 자유를 침해하고 사회 통제를 강화할 수 있습니다.

**6. 환경 문제:**

*   **막대한 에너지 소비:** 생성형 AI 모델을 학습시키고 운영하는 데는 엄청난 양의 에너지가 소모됩니다. 이는 탄소 배출량 증가로 이어져 기후 변화를 가속화할 수 있습니다.

**7. 악의적인 사용 및 사이버 공격:**

*   **악성 코드 생성 및 사이버 공격:** 생성형 AI는 악성 코드를 생성하거나 사이버 공격을 자동화하는 데 사용될 수 있습니다. 이는 기업, 정부 기관, 개인의 정보를 탈취하고 시스템을 마비시키는 데 악용될 수 있습니다.

**8. 인간의 창의성 및 비판적 사고 능력 저하:**

*   **AI 의존성 심화:** 생성형 AI에 대한 의존도가 높아짐에 따라 인간의 창의성, 문제 해결 능력, 비판적 사고 능력이 저하될 수 있습니다.

**9. 윤리적 책임 문제:**

*   **AI의 책임 소재 불분명:** 생성형 AI가 잘못된 결정을 내리거나 피해를 발생시켰을 경우, 누구에게 책임을 물어야 하는지에 대한 명확한 기준이 없습니다.

이러한 부정적인 측면들을 해결하기 위해서는 기술 개발과 더불어 윤리적 가이드라인, 법적 규제, 사회적 합의가 필요합니다. 생성형 AI를 안전하고 유익하게 활용하기 위해서는 지속적인 관심과 노력이 필요합니다.


------------------------------------ 
사용자: 종료


In [ ]:
session_id = "conv-004"
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 😊\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='생성형 AI 부정적인 측면 알려주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='생성형 AI는 놀라운 가능성을 보여주지만, 동시에 여러 가지 부정적인 측면도 가지고 있습니다. 주요한 부정적 측면들을 자세히 알려드릴게요.\n\n**1. 허위 정보 및 가짜 콘텐츠 생성:**\n\n*   **가짜 뉴스 및 허위 정보 확산:** 생성형 AI는 매우 사실적인 텍스트, 이미지, 오디오, 비디오를 만들 수 있기 때문에 가짜 뉴스를 쉽게 생성하고 퍼뜨릴 수 있습니다. 이는 사회적 혼란과 잘못된 정보에 기반한 의사 결정을 야기할 수 있습니다.\n*   **딥페이크:** 특정 인물의 얼굴이나 목소리를 합성하여 조작된 콘텐츠를 만드는 딥페이크는 명예훼손, 사기, 정치적 선전 등 다양한 악용 사례를 만들 수 있습니다.\n\n**2. 일자리 감소 및 경제적 불평등 심화:**\n\n*   **자동화로 인한 실업:** 생성형 AI는 콘텐츠 제작, 데이터 분석, 고객 서비스 등 다양한 분야에서 인간의 역할을 대체할 수 있습니다. 이는 특정 직업군의 실업률 증가로 이어질 수 있습니다.\n*   **숙련 노동자와 비숙련 노동자 간 격차 심화:** AI 기술을 활용할 수 있는 숙련 노동자와 그렇지 못한 비숙련 노동자 간의 임금 격차가 커질 수 있습니다.\n\n**3. 저작권 및 지적 재산권 침해:**\n\n*   **저작권 침해 콘텐츠 생성:** 생성형 AI는 학습 데이터에 포함된 저작물을 무단으로 사용하여 새로운 콘텐츠를 생성할 수 있습니다. 이는 저작권자의 권리를 침해하고 창작 

In [ ]:
# Manual input for chat history if you want to add something usefule to chat history.
chat_history.add_user_message("what's your role?")
chat_history.add_ai_message("I'm a helpful assistant")

chat_history.messages

[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요!  무엇을 도와드릴까요?\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국주변의 국제정세에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2025년 3월 19일 현재 한국 주변의 국제 정세는 매우 복잡하고 유동적입니다.  몇 가지 주요 요소들을 살펴보겠습니다.\n\n**1. 미중 전략 경쟁 심화:** 미국과 중국 간의 전략적 경쟁은 지속적으로 심화되고 있으며, 이는 한반도를 포함한 동아시아 지역에 큰 영향을 미치고 있습니다. 양국은 경제, 기술, 군사 등 다방면에서 경쟁을 벌이고 있으며, 이러한 경쟁은 지역 안보와 안정에 불확실성을 더하고 있습니다.  특히, 대만 문제는 미중 갈등의 핵심 쟁점이며, 무력 충돌 가능성까지 제기되면서 지역 전체의 긴장을 고조시키고 있습니다.\n\n**2. 러시아-우크라이나 전쟁 장기화:** 러시아-우크라이나 전쟁의 장기화는 국제 질서의 불확실성을 심화시키고, 세계 경제에 부정적인 영향을 미치고 있습니다.  전쟁은 에너지 가격 상승, 공급망 붕괴 등을 야기하며, 한국 경제에도 상당한 부담을 주고 있습니다.  또한, 러시아와 우크라이나 전쟁은 북한과의 관계에도 영향을 미칠 가능성이 있으며, 북한의 추가적인 도발 가능성을 높일 수 있다는 우려도 존재합니다.\n\n**3. 북한의 핵 및 미사일 개발:** 북한은 핵무기와 미사일 기술을 계속 개발하고 있으며, 이는 한반도와 동북아시아 지역의 안보에 심각한 위협이 되고 있습니다. 북한의 핵 및 미사일 개발은 주변국들의 안보 우려를 증폭시키고 있으며,  국제사회의 대북 제재 강화로 이어지고 있습니다.  하지만 북한의 도발에도 불구하

## End of Notebook